# Data Cleaning and Preparation for Amazon Fashion Dataset

### Overview
This notebook documents the process of downloading, cleaning, analyzing, and preparing the Amazon Fashion dataset for further use, such as building a vector database. The dataset contains product information, including titles, ratings, features, descriptions, prices, and images. The goal is to optimize the dataset by removing unnecessary columns, handling missing data, and reducing its size to meet GitHub's 100MB file size limit.

### Steps and Workflow
- **Initial Setup**:
    - Imported necessary libraries (`pandas`, `matplotlib`, etc.).
    - Configured pandas options for better display of data.
- **Data Downloading and Loading**:
    - Downloaded the dataset from a specified URL and saved it locally as a compressed JSONL file.
    - Loaded the dataset into a pandas DataFrame.
- **Exploratory Data Analysis (EDA)**:
    - Inspected the structure and content of the dataset using `.head()`, `.columns`, and `.iloc`.
    - Analyzed the distribution of key columns like `main_category`, `average_rating`, and `rating_number`.
- **Data Cleaning**:
    - Dropped irrelevant or empty columns such as `bought_together`, `videos`, `main_category`, and `categories`.
    - Extracted and retained only the `MAIN` `HIGH_RES` image URLs.
    - Concatenated multi-part descriptions into a single string.
    - Removed unnecessary prefixes from image URLs to reduce file size.
- **Data Optimization**:
    - Converted data types (e.g., `float32`, `int32`, `category`) to save memory.
    - Randomly dropped 5% of the data to reduce the dataset size.
- **Data Export**:
    - Saved the cleaned and optimized dataset as a compressed CSV file.

### Main Findings
- The dataset contained several columns with missing or irrelevant data:
    - `price` was missing for the majority of products.
    - `bought_together` and `categories` were fully null and dropped.
    - `main_category` had no variation and was removed.
- The `features` and `details` columns provided valuable information about the products.
- All non-null image URLs followed a consistent format, allowing for efficient processing.
- Multi-part descriptions were concatenated to improve usability.

### Results
- The cleaned dataset contains 784,803 rows and 9 columns.
- The dataset size was reduced to below 100MB by:
    - Dropping unnecessary columns.
    - Optimizing data types.
    - Removing 5% of the data randomly.
- The final dataset was saved as a compressed CSV file (`products.csv`) for further use.

In [ ]:
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
import os
import requests
import json

In [2]:
# This function will help us later when handling columns of dtype list
def flatten_and_count(df, column):
    """
    Flatten a column of lists and count occurrences of each unique item.
    """
    # Flatten all category lists (filtering out non-lists or NaNs)
    flattened = [
        category for row in df[column] if isinstance(row, list) for category in row
    ]

    # Count occurrences
    category_counts = Counter(flattened)

    # Convert to DataFrame for better readability or export
    category_df = pd.DataFrame.from_dict(
        category_counts, orient="index", columns=["count"]
    ).sort_values(by="count", ascending=False)

    return category_df

In [ ]:
# Define the URL of the dataset and the local path where it will be saved
url = "https://mcauleylab.ucsd.edu/public_datasets/data/amazon_2023/raw/meta_categories/meta_Amazon_Fashion.jsonl.gz"
output_path = "./data/meta_Amazon_Fashion.jsonl.gz"

# Check if the file already exists at the specified path
file_exists = os.path.exists(output_path)

# Print a message based on whether the file exists or not
if file_exists:
    print(f"The file already exists at {output_path}")
else:
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    print(f"Created directories for the path {output_path}")
    print(f"No file found at {output_path}")

The file already exists at ./raw/meta_Amazon_Fashion.jsonl.gz


In [4]:
# Check if the file already exists
if not file_exists:
    # If the file does not exist, download it from the specified URL
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        # Write the file in chunks to avoid memory issues with large files
        with open(output_path, "wb") as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        print(f"File downloaded and saved to {output_path}")
    else:
        # Print an error message if the download fails
        print(f"Failed to download file. Status code: {response.status_code}")
else:
    # If the file already exists, print a message
    print(f"The file already exists at {output_path}. Skipped download.")

The file already exists at ./raw/meta_Amazon_Fashion.jsonl.gz. Skipped download.


In [5]:
# Read the downloaded file into a pandas DataFrame
df = pd.read_json(output_path, lines=True, compression="gzip")

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 826108 entries, 0 to 826107
Data columns (total 14 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   main_category    826108 non-null  object 
 1   title            826108 non-null  object 
 2   average_rating   826108 non-null  float64
 3   rating_number    826108 non-null  int64  
 4   features         826108 non-null  object 
 5   description      826108 non-null  object 
 6   price            50249 non-null   float64
 7   images           826108 non-null  object 
 8   videos           826108 non-null  object 
 9   store            799270 non-null  object 
 10  categories       826108 non-null  object 
 11  details          826108 non-null  object 
 12  parent_asin      826108 non-null  object 
 13  bought_together  0 non-null       float64
dtypes: float64(3), int64(1), object(10)
memory usage: 88.2+ MB


In [ ]:
# Check the first few rows of the DataFrame
df.head()

In [ ]:
# Display the columns of the DataFrame to understand its structure
df.columns

In [ ]:
# Deep dive into a random row
df.iloc[3].to_dict()

We dont need all images for our prototype so let's pick up 1 image only - which is the `MAIN` `HIGH_RES` version

In [ ]:
df["main_hi_res_image"] = df["images"].apply(
    lambda images: (
        next(
            (
                img.get("hi_res")
                for img in images
                if img.get("variant", "").lower() == "main"
            ),
            None,
        )
        if isinstance(images, list)
        else None
    )
)
df = df.drop("images", axis=1)

In [ ]:
# Create and display a dataframe that summarizes the products
pd.DataFrame(
    {
        "Column Name": df.columns,
        "Data Type": df.dtypes.values,
        "Non-Null Count": df.notnull().sum().values,
        "Null Count": df.isnull().sum().values,
    }
)

- We see that `price` is missing for majority of products.
- `bought_together` is fully null and can be dropped.


In [ ]:
df = df.drop("bought_together", axis=1)

In [ ]:
# Let's also remove videos as it won't be needed in our prototype
df = df.drop("videos", axis=1)

In [ ]:
# Check main category distribution
df["main_category"].value_counts(dropna=False)

In [ ]:
# There is no variation so we can safely drop this columns
df = df.drop("main_category", axis=1)

In [ ]:
# Check the distribution of the categories
flatten_and_count(df, "categories")

In [ ]:
# Categories are fully empty so it can be dropped
df = df.drop("categories", axis=1)

In [ ]:
# Do the same for features column
flatten_and_count(df, "features")

There are 211,142 unique categories with most common ones as Pull On closure, Hand Wash Only, Machine Wash, Button closure, Zipper closure.

In [ ]:
# Lets look into details columns
df["details_keys"] = df["details"].apply(lambda details: list(details.keys()) if isinstance(details, dict) else {})
flatten_and_count(df, "details_keys")

- We see there 544 types of details such as model number, whether its discontinued or not. This is valuable information so let's keep it.
- Vector database we are going to use (Weaviate) requires same schema for all dictionaries, which is not the case here.
- So let's convert this in text format.

In [ ]:
df["details"] = df["details"].apply(json.dumps)
df = df.drop("details_keys", axis=1)

In [ ]:
# Lets check how ratings are distributed

# Histogram for average rating
df["average_rating"].dropna().hist(bins=100)
plt.title("Distribution of Average Rating")
plt.xlabel("Rating")
plt.ylabel("Count")
plt.show()

# Histogram for number of ratings
df["rating_number"].dropna().hist(bins=100)
plt.title("Distribution of Rating Count")
plt.xlabel("Number of Ratings")
plt.ylabel("Count")
plt.yscale("log")  # optional: log scale for better visibility
plt.show()

In [ ]:
# Now let's check description which is suprisingly a list field (Expected a string)
df["description_length"] = df["description"].apply(
    lambda x: len(x) if isinstance(x, list) else 0
)

df["description_length"].value_counts().sort_index()

In [ ]:
# Let's check cases where there are multiple descriptions
df[df["description_length"] > 3].head()["description"].values

We can see that these are just parts of an overall description. Let's concatenate them with space in between list elements

In [ ]:
df["description"] = df["description"].apply(
    lambda x: " ".join(x) if isinstance(x, list) else x
)
df = df.drop("description_length", axis=1)

In [ ]:
# Lets also remove parent_asin as people usually don't search using ASIN numbers
# This is done in order to reduce the size of the dataset to below 100MB which is the limit by GitHub
df = df.drop("parent_asin", axis=1)

In [ ]:
# Set pandas options to display full text in each column
pd.set_option("display.max_colwidth", None)

# Display the first 20 rows of the "main_hi_res_image" column
df["main_hi_res_image"].dropna().head(20)

In [ ]:
# We observe that URL always starts with "https://m.media-amazon.com/images/I/"
# (Does not necessarily end with ".jpg" as there are .png examples)
# Lets remove this prefix and add it later when we need to display the image
# This will help us reduce the size of the dataset

# Before that, let's validate that all URLs indeed start with this prefix
# Define the expected format
expected_format = r"^https://m\.media-amazon\.com/images/I/.*$"

# Count rows matching the format
matching_rows = df["main_hi_res_image"].str.match(expected_format, na=False).sum()

# Count rows not matching the format
non_matching_rows = df["main_hi_res_image"].notna().sum() - matching_rows

matching_rows, non_matching_rows

Indeed, all non-null image links start with the "https://m.media-amazon.com/images/I/" and end with ".jpg". Let's remove these strings from the column

In [ ]:
# Remove only the prefix from the URLs
df["main_hi_res_image"] = df["main_hi_res_image"].str.replace(
    r"^https://m\.media-amazon\.com/images/I/", "", regex=True
)

## Optimize df and save

In [ ]:
# Optimize data types
df["average_rating"] = df["average_rating"].astype("float32")
df["rating_number"] = df["rating_number"].astype("int32")
df["price"] = df["price"].astype("float32")
df["store"] = df["store"].astype("category")

In [ ]:
# At this stage, we can save the DataFrame to a CSV file in compressed format
# However, file size would still be above 100 MB which is the permitted size by GitHub
# Lets remove 5% of data randomly and save the remaining data in order to reduce the size

# Save the DataFrame with compression
df_sample = df.sample(frac=0.95, random_state=42)  # Drop 5% of rows randomly
df_sample.to_csv("./data/products.csv", index=False, compression="zip")

In [ ]:
# Check whether the size of saved table is below 100MB
file_size = os.path.getsize("./products.csv")
print(f"Size of './data/products.csv': {file_size / (1024 * 1024):.2f} MB")

- With removing unnecessary rows, optimizing dtypes, and dropping 5% of data, we were able to save the data with a size below 100MB
- The resulting data file can be used to build the Vector Database